In [ ]:
# 1. 최초 한 번만 실행
# 1. Run for the first time only

!git clone https://github.com/alstn1339/yolov5-indoor-space-type-classification

%cd yolov5-indoor-space-type-classification
%pip install -r requirements.txt  # install

In [2]:
# 잘못 설치 시 삭제용
# For deletion in case of incorrect installation
import shutil

shutil.rmtree('/content/yolov5-indoor-space-type-classification')

In [ ]:
#현재 작업 경로 확인, 작업 경로가 /content/yolov5-indoor-space-type-classification이어야 함.
#Check the current work path, the work path must be /content/yolov5-indoor-space-type-classification.
import os

print('현재 작업 경로:', os.getcwd())
os.chdir('/content/yolov5-indoor-space-type-classification')
print('변경된 작업 경로:', os.getcwd())

In [ ]:
# 2. 식별할 이미지 삽입 (이미지 별도 업로드)
# 2. Insert image to identify (Image uploaded separately)

!python detect.py --source '/content/yolov5-indoor-space-type-classification/test.jpg'
#display.Image(filename='runs/detect/exp/test.jpg', width=600)

In [8]:
# 3. 3차원 최종 코드 - 한 번만 구동
# 3. 3D Final code - Run Only Once

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pickle
import os

def save_cluster_data3(cluster_data, filename="cluster_data.pkl"):
    with open(filename, "wb") as file:
        pickle.dump(cluster_data, file)

def load_cluster_data3(filename="cluster_data.pkl"):
    with open(filename, "rb") as file:
        return pickle.load(file)

def perform_clustering3(csv_files, eps, min_samples):
    cluster_data = []
    all_data = []

    for file in csv_files:
        data = pd.read_csv(file)
        vectors = data.values
        all_data.append(vectors)
        scaler = StandardScaler()
        vectors_scaled = scaler.fit_transform(vectors)

        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(vectors_scaled)

        core_samples = vectors_scaled[dbscan.core_sample_indices_] if len(dbscan.core_sample_indices_) > 0 else []
        cluster_data.append((core_samples, scaler, dbscan))

    all_data_combined = np.vstack(all_data)
    pca = PCA(n_components=3)
    pca.fit(all_data_combined)

    # Save the cluster data and PCA model
    save_cluster_data3((cluster_data, pca), "cluster_data.pkl")

def process_new_vector3(new_vector, show_3d_plot=True, show_distance_table=True, show_closest_csv=True):
    # CSV 파일 경로 리스트
    csv_files = ["csvdata/Living_room_data.csv", "csvdata/Kitchen_data.csv", "csvdata/Library_data.csv", "csvdata/Bedroom_data.csv", "csvdata/Bathroom_data.csv"]  # 실제 파일 경로로 변경하세요.

    if not os.path.exists("cluster_data.pkl"):
        perform_clustering3(csv_files, eps=0.5, min_samples=5)

    cluster_data, pca = load_cluster_data3("cluster_data.pkl")

    # 새로운 벡터를 PCA 공간에 투영
    new_vector_pca = pca.transform([new_vector])[0]

    # 모든 데이터를 PCA 공간으로 변환합니다.
    all_data_pca = []
    for core_samples, scaler, _ in cluster_data:
        if len(core_samples) > 0:
            all_data_pca.append(pca.transform(scaler.inverse_transform(core_samples)))
        else:
            all_data_pca.append([])

    # 새로운 벡터와의 거리 계산 및 가장 가까운 CSV 파일 찾기
    min_distance = np.inf
    closest_csv = None
    distance_table = []  # 거리 표

    for i, (core_samples, scaler, model) in enumerate(cluster_data):
        if len(core_samples) == 0:
            distance_table.append((csv_files[i], None))  # 핵심 샘플이 없는 경우
            continue

        new_vector_scaled = scaler.transform([new_vector])
        distances = pairwise_distances(new_vector_scaled, core_samples, metric="manhattan")
        distance = np.min(distances)

        # 거리 표에 추가
        distance_table.append((csv_files[i], distance))

        if distance < min_distance:
            min_distance = distance
            closest_csv = csv_files[i]

    # 3D 시각화 + 새로운 벡터 표시
    if show_3d_plot:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')  # 변경: 3D 서브플롯 추가

        colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
        for i, data_pca in enumerate(all_data_pca):
            if len(data_pca) > 0:
                ax.scatter(data_pca[:, 0], data_pca[:, 1], data_pca[:, 2], c=colors[i % len(colors)], label=csv_files[i].split('_')[0])  # 변경: 3D 데이터 포인트 추가

        # 새로운 벡터 표시
        #ax.scatter(new_vector_pca[0], new_vector_pca[1], new_vector_pca[2], c='black', marker='x', label='New Vector')  # 변경: 새로운 벡터의 3D 표시

        ax.set_title('3D PCA of Clusters')  # 변경: 타이틀 변경
        ax.legend()
        plt.show()

    # 거리 표 출력
    if show_distance_table:
        print("\nDistance Table:")
        print("CSV File\t\t\tDistance")
        for (csv_file, distance) in distance_table:
            distance_str = 'N/A' if distance is None else f'{distance:.2f}'
            print(f"{csv_file}\t{distance_str}")

    # 가장 가까운 CSV 파일 출력
    if show_closest_csv:
        print(f"\nThe new vector is closest to the clusters from: {closest_csv}")

    return closest_csv

# 처음 실행할 때 군집화 수행 및 데이터 저장
csv_files_initial3 = ["csvdata/Living_room_data.csv", "csvdata/Kitchen_data.csv", "csvdata/Library_data.csv", "csvdata/Bedroom_data.csv", "csvdata/Bathroom_data.csv"]
perform_clustering3(csv_files_initial3, eps=0.3, min_samples=2)

In [ ]:
# 4. 결과창
# 4. Results window

new_vector_example = pd.read_csv("room_data.csv").values[0]
process_new_vector3(new_vector_example, show_3d_plot=True, show_distance_table=True, show_closest_csv=True)